# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [ ]:
websites = pd.read_csv('../data/website.csv')

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for more exploration. Make sure to comment what you find!

In [ ]:
print(websites.shape)
websites.head()

In [ ]:
websites.dtypes

In [ ]:
websites.isna().sum()

In [ ]:
websites.Type.value_counts(normalize=True)

In [ ]:
"""
The shape of the dataset is (1781, 21) so we have enough rows and features to build a proper model. 

We have some categorical data that may require to be transform as dummies 
if we want to use a linear regression model. 
In addition, we may want to create a new column to calculate the time of the last WHOIS_UPDATE or WHOIS_REGATE. 
It would allow to transform this column into numerical data with a better purpose than dummies. 

We have some missing values that should be fixed. 

The column we want to predict would be the Type 0 or 1. Begnin would stand for 0 and malicious for 1. 

The interesting features all all the columns except for URL and Type (predicted column).

Finally we don't have a balanced dataset (87.9% of Type are begnin and 12.1% are malicious), 
so we should take that into account when building our model and test different paramaters to improve accuracy.

"""

#### Next, evaluate if the columns in this dataset are strongly correlated.

In class, we discussed that we are concerned if our dataset has strongly correlated columns because if this is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [ ]:
websites.corr()

In [ ]:
sns.heatmap(websites.corr());

In [ ]:
"""
After looking at correlation between features, I would remove the following columns:

- URL_LENGTH which is correlated with NUMBER_SPECIAL_CHARACTERS and this latter seems more appropriate 
to detect malicious website rather than the length of the URL

- SOURCE_APP_PACKETS, REMOTE_APP_PACKETS, SOURCE_APP_BYTES, APP_PACKETS which are correlated 
to TCP_CONVERSATION_EXCHANGE

- REMOTE_APP_BYTES which is correlated to APP_BYTES

"""

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [ ]:
# I copy the original dataframe in case of errors
webs = websites.copy()

In [ ]:
webs = webs.drop('URL_LENGTH',axis=1)

In [ ]:
webs = webs.drop('SOURCE_APP_PACKETS',axis=1)

In [ ]:
webs = webs.drop('REMOTE_APP_PACKETS',axis=1)

In [ ]:
webs = webs.drop('SOURCE_APP_BYTES',axis=1)

In [ ]:
webs = webs.drop('APP_PACKETS',axis=1)

In [ ]:
webs = webs.drop('REMOTE_APP_BYTES',axis=1)

In [ ]:
sns.heatmap(webs.corr());

# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [ ]:
webs.isna().sum()

If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [ ]:
(webs.isna().sum()/webs.shape[0])*100

In [ ]:
# Dropping content_lenght that has more than 45% of missing values
# The column does not seem so relevant to detect malicious websites either so it is okay to drop it.

webs = webs.drop('CONTENT_LENGTH',axis=1)

In [ ]:
# Checking the rows with NaN values 

webs[(webs.SERVER.isna())|(webs.DNS_QUERY_TIMES.isna())]

In [ ]:
# Finding a value that could be used to fill the
fillna_server = webs[(webs.WHOIS_COUNTRY=='US')].SERVER.mode()
fillna_server[0]

In [ ]:
webs.SERVER = webs.SERVER.fillna(fillna_server[0])
webs.iloc[1306,:]

In [ ]:
webs.DNS_QUERY_TIMES.value_counts()

In [ ]:
# DNS query times might be a strong indicator of malicious website, I would drop this row to avoid 
# biases and misleading the model

webs = webs.drop(1659, axis=0)

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [ ]:
# Examine missing values in each column
webs.isna().sum()

# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [ ]:
webs.WHOIS_COUNTRY.value_counts()

In [ ]:
dict_country = {'United Kingdom':'GB',"[u'GB'; u'UK']":'GB','Cyprus':'CY'}
webs.loc[webs.WHOIS_COUNTRY.isin(['United Kingdom',"[u'GB'; u'UK']",'Cyprus']),'WHOIS_COUNTRY'] = webs[webs.WHOIS_COUNTRY.isin(['United Kingdom',"[u'GB'; u'UK']",'Cyprus'])].WHOIS_COUNTRY.map(dict_country)


In [ ]:
webs.WHOIS_COUNTRY = [i.upper() for i in webs.WHOIS_COUNTRY.values]

In [ ]:
webs.WHOIS_COUNTRY.nunique()

Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [ ]:
webs.WHOIS_COUNTRY.value_counts().plot(kind='bar',figsize=(17,5));

#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [ ]:
top_10 = webs.WHOIS_COUNTRY.value_counts().head(10).index
webs.WHOIS_COUNTRY = webs.WHOIS_COUNTRY.apply(lambda x: x if x in top_10 else 'OTHER')

Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [ ]:
webs = webs.drop(columns=['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE'])

In [ ]:
webs.shape

# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [ ]:
webs.dtypes

#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [ ]:
webs = webs.drop('URL', axis=1)

#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [ ]:
webs.CHARSET.nunique()

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [ ]:
webs.SERVER.value_counts().head(20)

![Think Hard](../images/think-hard.jpg)

In [ ]:
"""
We should rid off the version of the server and only keep the general name of the server.

"""


Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [ ]:
def replace_server(x):
    if x.count('Microsoft')>0:
        return 'Microsoft'
    elif x.count('Apache')>0:
        return 'Apache'
    elif x.count('nginx')>0:
        return 'nginx'
    else:
        return 'Other'

webs.SERVER = webs.SERVER.apply(lambda x: replace_server(x))

In [ ]:
# Count `SERVER` value counts here
webs.SERVER.value_counts()

In [ ]:
webs.dtypes

OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [ ]:
# Your code here
website_dummy = pd.get_dummies(webs,columns=['WHOIS_COUNTRY','SERVER','CHARSET'],drop_first=True)

Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [ ]:
# Your code here
website_dummy.dtypes

# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [ ]:
from sklearn.model_selection import train_test_split

X = website_dummy.drop('Type',axis=1)
y = website_dummy.Type

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .20, random_state=8, stratify=website_dummy.Type)


In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [ ]:
model.fit(X_train,y_train)

finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred_LR = model.predict(X_test)
print(accuracy_score(y_test,y_pred_LR))
confusion_matrix(y_test,y_pred_LR)

What are your thoughts on the performance of the model? Write your conclusions below.

In [ ]:
"""
The performances seem pretty good with 92.7% of data that have been predicted right. 

We can see only 5 false positive (5 websites were predicted malicious but are not) so the damages
are not so important, we can accept this level of errors. 

"""


#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=3)
model_knn.fit(X_train,y_train)

To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [ ]:
y_pred = model_knn.predict(X_test)
print(accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [ ]:
model_knn_5 = KNeighborsClassifier(n_neighbors=5)
model_knn_5.fit(X_train,y_train)
y_pred = model_knn_5.predict(X_test)
print(accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [ ]:
"""
We can see a slightly improvement when increasing k to 5, the accuracy score increased by 0.008

Also we can see it reduced the number of false positive from 12 to 5. It should be considered 
as a positive improvement because we will reduce the number of damage to website that are actually not malicious.

"""



# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [ ]:
from sklearn.preprocessing import RobustScaler

transformer = RobustScaler()
transformer.fit(X_train)

In [ ]:
X_train.columns.shape

In [ ]:
X_train_scaled = transformer.transform(X_train)
pd.DataFrame(X_train_scaled,columns=X_train.columns)

In [ ]:
X_test_scaled = transformer.transform(X_test)

In [ ]:
model_scaled = LogisticRegression()
model_scaled.fit(X_train_scaled,y_train)

In [ ]:
y_pred_scaled = model_scaled.predict(X_test_scaled)
print(accuracy_score(y_test,y_pred_scaled))
confusion_matrix(y_test,y_pred_scaled)

In [ ]:
print(accuracy_score(y_test,y_pred_LR))
confusion_matrix(y_test,y_pred_LR)

In [ ]:
"""
The results are almost the same but we can't see any improvements. The results without scaled data is better.
"""